# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from numpy.linalg import inv

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Мы можем заключить что все данные числовые и представлены в подходящих типах данных, поэтому в будущем эти данные можно будет преобразовать в матричный вид. 

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Ответьте на вопрос и обоснуйте решение.
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

    a. Изменится. Приведите примеры матриц.
    b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменится

**Обоснование:** Установим что у нас есть первоначальный набор данных в виде матрицы (только features без target) X и обратимая матрица P (значит существует матрица P^-1) на которую производим матричное умножение. 
Для нашего объяснения нам понадобятся следующие формулы.

Первоначальная формула обучения:

$$ w = (X^T X)^{-1} X^T y $$ 

Первоначальная формула предсказания:

$$ a = Xw $$

Несколько свойств матричных операций:

$$ AA^{-1}=A^{-1}A=E $$ (E - это единичная матрица)

$$ AE=EA=A $$

$$ (AB)^T=B^TA^T $$ (т.е. A и B меняются местами)

$$ (AB)^{-1}=B^{-1}A^{-1} $$ (при условие что и A и B квадратные, так как только у них существуют обратные матрицы)
$$ (ABC)^{-1}=C^{-1}B^{-1}A^{-1} $$ (при условие что все три матрицы квадратные, так как только у них существуют обратные матрицы)

Представим новую формулу обучения, где каждая X умножится на P (а “y” не изменится, так как target выведен отдельно):
$$ w_{new} = ((XP)^T (XP))^{-1} (XP)^T y $$

Проведём постепенное раскрытие скобок:
$$ w_{new} =  (P^TX^TXP)^{-1} P^T X^T y $$
Замечаем что в первой скобке:
$$ (P^TX^TXP)^{-1} $$
Стоят три квадратные матрицы: Р по условию обратная (а значит квадратная), а при умножение траспонированной матрицы X на саму эту матрицу X, получится квадратная матрица (так как стороны будут равны ширине первоночальной матрицы). Поэтому мы можем ракрыть скобки:
$$ w_{new} =  P^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y $$
Последние две траспонированные P можно сократить, так как они дают E. Остаётся такая формула:
$$ w_{new} =  P^{-1} (X^TX)^{-1} X^T y $$
Три последних X и “y” представляют собой формулу w. 
Поэтому формула будет выглядеть так:
$$ w_{new} =  P^{-1} w $$

Напомним первоначальную формула предсказания:
$$ a=Xw $$
Запишем формулу предсказания для новой матрицы:
$$ a_{new}=XP w_{new} $$
Вставим туда новое w которое мы нашли выше:
$$ a_{new}=XP  P^{-1} w $$
Сократим Р:
$$ a_{new}=Xw $$
Получается что новое предсказание равно старому. Значит мы ничего не потеряли, и самое главное не приобрели ничего нового для вычисления предсказаний линейной регрессии. Но сами данные защитили. 

## Алгоритм преобразования

Мы теоретически выяснили, если умножить исходную матрицу данных на обратимую матрицу, то для предсказаний линейной регрессии не будет ни потерь ни приобретений, но данные мы защитим. Теперь проверим это практически. Нам нужно сохранить target без изменений, поэтому его нам нужно будет сразу отделить от данных. Соответственно features будет состоять из 4 колонок. Значит обратимую матрицу надо будет сделать 4x4, так как обратимые матрицы должны быть квадратными. Также надо будет проверить наши преобразования вычислением линейной регрессии, поэтому первоначальную матрицу данных до умножения нужно будет сохранить. 

Поэтому наш алгоритм действий будет следующий:

1. Разделим данные на features и target, а их на обучающую и тестовую выборки.
2. Создадим матрицу 4x4.
3. Проверим её на обратимость. 
4. Преобразуем выборки features в матрицы, в отдельные переменные.
5. Проведём умножение матриц из 4-го пункта на матрицу из 2-го пункта. 
6. Создадим модели и посчитаем метрику R2 на первоначальных данных и на преобразованных матричным умножением. 
7. Подведём итог.

## Проверка алгоритма

In [5]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

In [6]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [7]:
p = np.random.randint(7, size=(4, 4))

In [8]:
# Создадим обратную матрицу 
p_inv = inv(p)

#Проверим обратимость, если выдаст True, то обратима
print(np.allclose(np.dot(p, p_inv), np.eye(4)))
print(np.allclose(np.dot(p_inv, p), np.eye(4)))

True
True


In [9]:
features_train_p = features_train.values
features_test_p = features_test.values

In [10]:
features_train_p = features_train_p @ p
features_test_p = features_test_p @ p

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
r2 = r2_score(target_test, predicted_test)

print('R2 на первоночальных данных:', r2)

R2 на первоночальных данных: 0.43522757127026546


In [12]:
model_p = LinearRegression()
model_p.fit(features_train_p, target_train)
predicted_test_p = model_p.predict(features_test_p)
r2_p = r2_score(target_test, predicted_test_p)

print('R2 на преобразованных данных:', r2_p)

R2 на преобразованных данных: 0.4352275712702721


**Итог**

Нам были предоставлены данные, в которых должно путём преобразований данных защитить персональную информацию, но одновременно сделать данные пригодными для работы модели линейной регрессии. 

Сами данные не содержали пропусков, а их тип данных был приемлем для преобразований. 

Для преобразования данных нами был предложен способ матричного умножения данных. То есть первоначальные данные делились на features и target. Потом features преобразовывался в матрицу и умножался на обратимую квадратную рандомную матрицу. Во втором пункте, нами была обоснована теоретическая база для такого преобразования, так как в результате теоретических расчётов предсказания модели линейной регрессии должны совпадать на и на первичных и на преобразованных данных.  

Нами также была проведена проверка наших теоретических выводов. Построены модели линейной регрессии на первичных и на преобразованных данных. Метрика качества R2 почти полностью совпала на этих моделях, а значит для модели линейной регрессии нет потерь если она будет использовать преобразованные данные. Конфиденциальная же информация в данных получит защиту. 